# The least profitable rubber price at the county level 

Notebook illustrates how to compute the least profitable rubber price ($/kg) at the county level in Laos, Myanmar, and Vietnam.

The cost of natural rubber contributed from:
- the costs of rubber plantation establishment and management;
- the costs of natural rubber harvest;
- the costs of natural rubber transportation;
- the social-economic factors include the social discount rate, least hourly labor cost and the average rubber productivity of rubber plantations in each countries, etc.


### The minimum price for sustainable livelihood of rubber smallholders

To maintain a sustainable livelihood for smallholding rubber farmers in East Asia, the economic returns of natural rubber products must at least equal the overall costs of establishing, managing and harvesting the rubber plantation.  In this study, we calculated a minimum price for natural rubber products, again at the county level, at which rubber cultivation would be sustainable.  Therefore, if the actual market price of the product is higher than the minimum price, the smallholders will profit. However, if the actual price is lower than the minimum price, rubber cultivation is not a sustainable livelihood for them.  The minimum price is the average minimum price over 25 years of rubber plantation lifespan. The county-level minimum rubber prices were calculated in the eight study countries.
To be able to calculate the minimum price of natural rubber products, an economic concept was introduced here: the net present value (NPV). NPV represents a time series of cash flow for investment, for which future revenue is discounted to represent opportunity costs in relation to the present value.  It is an efficient investment while NPV is positive, but it is not while the NPV is negative.  

It’s written in a formula as: 

$NPV = \sum \limits _{t=0} ^ {25} (B_{t} - C_{t}) / (1 + r ) ^t $

Bt is the gross economic returns for the plantation in our study, it is equal to the rubber yield multiply by the price at year t. 
Ct represents the overall cost calculated above. 
r is the discount rate. Simply, discount rate tells the expected return rate for the individual investor. While the investor uses higher discount rate, which means he/she expected higher return rate from the investment. In this study we use social discount rate, which is the expected return rate for the society. The social discount rate is higher in the less developed countries, which means the investors expect to have their investment cash back sooner than the investors who invests in the more developed countries or regions.  A higher social discount rate also indicates the less public sector investment for the country. As less developed countries in tropical Asia and rubber has been listed as a tree cash crop for poverty alleviation, e.g. Laos, Myanmar, and Cambodia, have a higher social discount rate compare to other study countries. 
The minimum price could be calculated when Bt - Ct = 0, which means that the minimum price is the price that could cover the overall cost for having a rubber plantation. In this study it was calculated as: 

$P_{rubber} =  \sum \limits _{t=0} ^ {25} (C_{t} / (1+r) ^ t / P $



In [ ]:
import os
from os import path as op
import pandas as pd
import numpy as np 
import json 

In [ ]:
data_path = "../../admin_data"
if os.getcwd() == "/mnt":
    data_path = "/mnt/admin_data"
print(data_path)

In [ ]:
csvs = sorted([op.join(data_path, csv) for csv in os.listdir(data_path) if csv.endswith('.csv')])
csvs

In [ ]:
def rubber_costs(country_prof, cost_csv):
    """
    Compute labor and social discount
    """
    costs_df = pd.read_csv(cost_csv)
    # adding "labor cost" 6 hours work per day
    costs_df['labor_cost'] = costs_df['Total labor days'] * 6 * country_prof['Minimum wage (USD/day)']
    # adding "social discount" for 25 years of rubber planation life-span
    costs_df['social_discount'] = [(1+ country_prof['Social discount rate']) ** t for t in range(1, 26)]
    return costs_df

In [ ]:
def least_profitable_price(country_csv, costs_country, country_prof):
    """
    Compute the least profitable rubber price from:
    - the costs;
    - social discount;
    - over a rubber planation's 25 year lifespan;
    - at admin level, e.g. county, province and county.
    """
    # the factor to transport rubber product to the market, hard to get to market the cost is higher
    market_acc_factor = 2
    # rubber factor to reflect more rubber planations within an admin area the rubber cost is lower
    rf = 0.05
    # orginal cost is saved as ha, convert to km2 is 100
    ha2km2 = 100 
    # the life span of a rubber planation is 25 
    plantation_lsp = 25
    # rubber plantation starts to make profit from year of 7th
    production = 25 -6 
    
    country_df = pd.read_csv(country_csv) 
    country_df["rubber_2020"] = country_df["rubber_2020"].apply(lambda x: json.loads(x.replace("'",'"')))
    country_df["market_acc_stat"] =country_df["market_acc_stat"].apply(lambda x: json.loads(x.replace("'",'"')))
    # calculate rubber plantation mean area for the admin level 
    mean_pa=np.nanmean([row.get("255.0", np.nan) for row in country_df["rubber_2020"]])

    print(mean_pa)
    
    # looping through each county from the contry csv
    # each row represent each county
    # compute county level cost, and total rubber productivity
    lpps = []
    for index, row in country_df.iterrows():
        tc = sum(costs_country['Transport expenses'])
        tlc = sum(costs_country['Transport of latex'])
        
        cunty_tc_sum = sum(tc* float(k)* market_acc_factor * (float(v) / ha2km2) for k, v in row['market_acc_stat'].items())
        cunty_tlc_sum = sum(tlc* float(k)* market_acc_factor * (float(v) / ha2km2) for k, v in row['market_acc_stat'].items())
        #sum of total cost of"Eletricity, water, tools, and other cost" over 25 year
        county_wet = sum(costs_country['Eletricity, water, tools, and other cost'])
        # sum of total cost of "Chemical using expenses"
        county_chem = sum(costs_country['Chemical using expenses'])
        # sum of labor cost 
        county_labor = sum(costs_country['labor_cost'])
        # sum of social discount 
        county_sd = sum(costs_country['social_discount'])
        # total cost over 25 years at county level for all rubber planations
        try:
            costs = (((cunty_tc_sum + cunty_tlc_sum + county_wet + county_chem + county_labor) *ha2km2/ (row['rubber_2020']['255.0'] / mean_pa) *rf)
                     /county_sd) * row['rubber_2020']['255.0'] * plantation_lsp
            productivity = laos['Average productivity (kg/ha)'] * production * ha2km2 * row['rubber_2020']['255.0'] 
            lpp = round(costs/productivity, 3)
        except:
            lpp = np.nan
        print(row['shapename'], lpp)
        lpps.append(lpp)
    
    country_df['rubber_least_profitable_price'] = lpps
    country_df.to_csv(country_csv)
    return country_df

### Country minimum wage, average rubber productivity and social discount

In [ ]:
laos = dict()
laos['Minimum wage (USD/day)'] = 5
laos['Average productivity (kg/ha)'] = 780
laos['Social discount rate'] = 0.14

In [ ]:
vietnam =  dict()
vietnam['Minimum wage (USD/day)'] = 5
vietnam['Average productivity (kg/ha)'] = 754
vietnam['Social discount rate'] = 0.13

In [ ]:
myanmar =  dict()
myanmar['Minimum wage (USD/day)'] = 6
myanmar['Average productivity (kg/ha)'] = 1717
myanmar['Social discount rate'] = 0.11

### Cost factors for establishing, management and transportation rubber plantations

The cost is in $ per hectare

In [ ]:
cost_csv = op.join(data_path,"../scripts/rubber_least_profitable_price/average_costs_rubber_planations.csv")
costs_df = pd.read_csv(cost_csv)
costs_df.head()

### Admin data for the platform

In [ ]:
laos_admin2 = pd.read_csv(op.join(data_path,'la-adm2.csv'))
laos_admin2.head()

In [ ]:
csvs

## Laos

In [ ]:
rcosts_laos = rubber_costs(laos, cost_csv)
rcosts_laos.head()

#### County level the least profitable rubber price

In [ ]:
laos_county_lrpp = least_profitable_price(op.join(data_path,'la-adm2.csv'), rcosts_laos, laos)

#### Province level 

In [ ]:
laos_pro_lrpp = least_profitable_price(op.join(data_path,'la-adm1.csv'), rcosts_laos, laos)

#### Country level

In [ ]:
laos_country_lrpp = least_profitable_price(op.join(data_path,'la-adm0.csv'), rcosts_laos, laos)

## Vietnam

In [ ]:
rcosts_vn = rubber_costs(vietnam, cost_csv)
rcosts_vn.head()

In [ ]:
vn_county_lrpp = least_profitable_price(op.join(data_path,'vn-adm2.csv'), rcosts_vn, vietnam)

In [ ]:
vn_prov_lrpp = least_profitable_price(op.join(data_path,'vn-adm1.csv'), rcosts_vn, vietnam)

In [ ]:
vn_country_lrpp = least_profitable_price(op.join(data_path,'vn-adm0.csv'), rcosts_vn, vietnam)

###  Myanmar

In [ ]:
rcosts_mm = rubber_costs(myanmar, cost_csv)
rcosts_mm.head()

In [ ]:
mm_county_lrpp = least_profitable_price(op.join(data_path,'mm-adm2.csv'), rcosts_mm, myanmar)

In [ ]:
mm_prov_lrpp = least_profitable_price(op.join(data_path,'mm-adm1.csv'), rcosts_mm, myanmar)

In [ ]:
mm_country_lrpp = least_profitable_price(op.join(data_path,'mm-adm0.csv'), rcosts_mm, myanmar)

In [ ]:
mm_prov_lrpp.head()